In [8]:
#setup

import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
from Silanizer.Grafter.grafter import Grafting
import subprocess
from natsort import natsorted
from glob import glob
import json

GMXSOURCE = f"/scratch/project_465000467/FABIO/modules.sh"
COLORS = {"N1B":"xkcd:light gray","N1L":"xkcd:light gray","N1L1":"xkcd:light gray","DMS":"xkcd:amber","N1P":"xkcd:light gray","W":"xkcd:water blue","TW":"xkcd:water blue","SW":"xkcd:water blue","TOLU":"xkcd:algae green"}
NAMES = ["DMS","N1P","N1B","N1L","N1L1","W","SW","TW","TOLU"]
MOLDEFS = {"water":[["TW","","TW_box.gro"],["SW","TW_box.gro","SW_TW_box.gro"],["W","SW_TW_box.gro","SW_TW_W_box.gro"]], "toluene":[["TOLU","","TOLU_box.gro"]]}
NSOLV = {"K0,0.1": 10000, "K1,0.1":10000, "K2,0.1":10000, "K5,0.1":10000,
         "K0,0.2": 10000, "K1,0.2":10000, "K2,0.2":10000, "K5,0.2":10000,
         "K0,0.3": 10000, "K1,0.3":20000, "K2,0.3":20000, "K5,0.3":30000,
         "K0,0.6": 10000, "K1,0.6":20000, "K2,0.6":20000, "K5,0.6":30000,
         "K0,1.0": 10000, "K1,1.0":20000, "K2,1.0":20000, "K5,1.0":30000}
MASS = {"water":162, "toluene":126}
DENSITY = {"water":1, "toluene":0.866}         
NATOMS = {"water":3, "toluene":1}
TOPOL = "vacuum.top"
ROOT = "/scratch/project_465000467/FABIO/PDMS"


In [11]:
#assemble system and solvate
def calc_nmols(rho, v, mass):
        amu_to_g = 1.66054e-24
        return int(rho*(v*1.0e-21)/(mass*amu_to_g))

SAMPLES = ["K5"]
DENS = [1.0]
RUNS = ["run3"]
SOLV = "water"
MOLDEFS = {"water":[["TW","","TW_box.gro"],["SW","TW_box.gro","SW_TW_box.gro"],["W","SW_TW_box.gro","SW_TW_W_box.gro"]], "toluene":[["TOLU","","TOLU_box.gro"]]}
NSOLV = {"K0": 10000, "K1":10000, "K2":10000, "K5":10000}
SYSTEMS = [[s,d,r] for s in SAMPLES for d in DENS for r in RUNS]
TOPOL = "vacuum.top"
POLY = "Polydisperse"

for sys in tqdm.tqdm(SYSTEMS):
        SAMPLE, DEN, RUN = sys

        print("Setting up system.\nSample: ", SAMPLE, "\nDensity: ", DEN, "\nRun: ", RUN, "\nSolvent: ", SOLV, "\n")
        startFolder = f"{ROOT}/{POLY}/{SAMPLE}/{DEN}/{RUN}/{SOLV}/min"
        systemFolder = f"{ROOT}/{POLY}/{SAMPLE}/{DEN}/{RUN}/vacuum/nvt"

        subprocess.run(f"mkdir -p {startFolder}", shell = True, executable="/bin/bash")
        groFile = natsorted(glob(f"{systemFolder}/*part*.gro"))[-1]

        defs = np.array(MOLDEFS[SOLV])
        for mol in defs[:,0]:
                subprocess.run(f"cp {ROOT}/start_files/{mol}.gro {startFolder};",
                        shell = True, executable="/bin/bash")                                            
        subprocess.run(f"cp {groFile} {startFolder}/start.gro;"+
                f"cp {ROOT}/start_files/WALL.gro {startFolder};"+
                f"cp {systemFolder}/{TOPOL} {startFolder}/;"+
                f"cp -r {systemFolder}/itps {startFolder}/;"+
                f"cp -r {ROOT}/start_files/itps/* {startFolder}/itps/",
                shell = True, executable="/bin/bash")

        u = mda.Universe(f"{startFolder}/start.gro")
        atoms = u.select_atoms("all").positions
        pbcBox = u.dimensions[:3].astype(float)
        zmax = np.max(atoms[:,2])
        wallPosition = zmax+200
        pbcBox = [pbcBox[0], pbcBox[1], wallPosition+300]

        inputs = {      "folder": f"{ROOT}/{POLY}/{SAMPLE}/{DEN}/{RUN}/{SOLV}/min",
                        "blocks": [ "start.gro" , "WALL.gro" ],
                        "positions": [ [0,0,0] , [0,0,wallPosition] ],
                        "box dimensions": [ pbcBox[0], pbcBox[1], pbcBox[2], 90, 90, 90 ],
                        "transforms": False,
                        "name": SAMPLE,
                        "out name": "assembled.gro"     }

        MySys = Grafting.NewSystem(root=startFolder)
        MySys.read_inputs_assembler(inputs)
        MySys.run_assembler(topol=TOPOL, molnames=["WALL"])

        boxSize = np.array([pbcBox[0], pbcBox[1], wallPosition - zmax - 20])*.1
        positions = np.array([[0,pbcBox[0]],[0,pbcBox[1]],[zmax+20, wallPosition-20]])*.1
        pbcBox = np.array(pbcBox)*.1
        numMols = calc_nmols(DENSITY[SOLV], np.prod(boxSize),MASS[SOLV])

        for data in defs: 
                mol, into, out = data
                MySys.solv_box(MySys.folder, GMXSOURCE, f"{out}", f"{mol}", boxSize, positions, pbcBox, numMols, into=into)
                if len(into) > 1:
                        print(f"Inserted {numMols} of {mol} into {into}, filename {out}.")
                else:
                        print(f"Created box with {numMols} of {mol}, filename {out}.")
            
        inFile = "assembled.gro"
        solvation = "centered_"+defs[:,2][-1]
        outFile = "solvated_start.gro"
        numMolsTotal = numMols*NATOMS[SOLV]
        MySys.solvate(GMXSOURCE, f"{startFolder}/{inFile}", solvation, outFile, TOPOL, pbcBox, numMolsTotal)
        print(f"\nSolvated {inFile} with {solvation}, filename {outFile}.\n")


        MySys.plot_system(colors=COLORS,names=NAMES)


        jobName = f"{SAMPLE}_{DEN}_{RUN}_min"
        subprocess.run(f"cd {startFolder}; cp {ROOT}/scripts_PDMS/run_min* .;"+
                f"source {GMXSOURCE};"+
                f"printf 'a N1L1\na DMS\nq\n' | gmx make_ndx -f {outFile} -o index.ndx > make_ndx.log 2>&1;"+
                f"sed -i 's/freezegrps              = .*/freezegrps              = BULK N1L1 LAY WALL/' run_min.mdp;"+
                f"sed -i 's/freezedim               = .*/freezedim               = Y Y Y Y Y Y Y Y Y Y Y Y/' run_min.mdp;"+
                f"gmx grompp -p {TOPOL} -c {outFile} -f run_min.mdp -o minimized.tpr -n index.ndx -maxwarn 2 > grompp.log 2>&1;"+
                f"sbatch --job-name=\"{jobName}\" run_min.sh minimized;",
                shell=True, executable="/bin/bash")

  0%|          | 0/1 [00:00<?, ?it/s]

Setting up system.
Sample:  K5 
Density:  1.0 
Run:  run3 
Solvent:  water 



/scratch/project_465000467/FABIO/.venv/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))



name: K5
root: /scratch/project_465000467/FABIO/PDMS/Polydisperse/K5/1.0/run3/water/min
blocks: ['start.gro', 'WALL.gro']
positions: [[  0.           0.           0.        ]
 [  0.           0.         672.45999146]]
box: [501.1793212890625, 98.99020385742188, 972.4599914550781, 90, 90, 90]
transformations: [None, None]
out name: assembled.gro


.*.*.*.*.*.*.*.*.*.*.*.
Starting assembling

name:  K5 
root:  None 
blocks:  ['start.gro', 'WALL.gro'] 
positions:  [[  0.           0.           0.        ]
 [  0.           0.         672.45999146]] 
box:  [501.1793212890625, 98.99020385742188, 972.4599914550781, 90, 90, 90] 
transformations:  [None, None] 
out name:  assembled.gro 

Finished assembling (:
.*.*.*.*.*.*.*.*.*.*.*.




KeyboardInterrupt: 